# mysql 테이블 생성 및 데이터 입력 방법

In [1]:
!pip install pymysql > /dev/null

from google.colab import files
uploaded = files.upload()
filename = list(uploaded.keys())[0]

Saving mysql.json to mysql.json


In [2]:
import json
with open(filename) as fp:
    config_str = fp.read()
config = json.loads(config_str)

In [3]:
config['database'] = 'mcdb'

In [5]:
import pymysql
conn = pymysql.connect(
    host=config['host'], 
    user=config['user'], 
    password=config['password'], 
    database=config['database'], 
    port=config['port']
)

In [7]:
cur = conn.cursor()
sql = "DESC users;"
cur.execute(sql)

5

In [8]:
for row in cur.fetchall():
    print(row)

('uid', 'varchar(20)', 'NO', 'PRI', None, '')
('pwd', 'char(44)', 'NO', '', None, '')
('uname', 'varchar(20)', 'NO', '', None, '')
('reg_date', 'datetime', 'YES', '', 'CURRENT_TIMESTAMP', 'DEFAULT_GENERATED')
('is_deleted', 'int', 'YES', '', '0', '')


## 테이블 생성

In [10]:
cur = conn.cursor()
sql_create_table = '''
    create table if not exists users ( 
        uid varchar(20) not null primary key, 
        pwd char(44) not null, 
        uname varchar(20) not null, 
        email varchar(40),
        reg_date datetime default current_timestamp, 
        is_deleted int default 0 
        );
    '''

In [12]:
cur.execute(sql_create_table)

0

In [13]:
cur = conn.cursor()
sql_insert = "INSERT INTO users(uid,pwd, uname) VALUES('admin','1234','관리자');"
cur.execute(sql_insert)

1

In [15]:
cur = conn.cursor()
sql_select = "select * from users;"
cur.execute(sql_select)
row = cur.fetchone()
row

('admin', '1234', '관리자', None, datetime.datetime(2021, 8, 25, 13, 28, 40), 0)

In [16]:
for row in cur.fetchall():
    print(row)


In [17]:
# 데이터를 하드 디스크에 쓰게 하는 명령
conn.commit()

- 여러건의 데이터 추가

In [24]:
sql_multi_insert= """
    Insert into users(uid, pwd, uname) values
    ('eskim', '1234','김은숙'),
    ('wjlee', '1234', '이우정');
"""



In [25]:
cur.execute(sql_multi_insert)
conn.commit()

- Placeholder

In [26]:


sql_insert_ph = "Insert into users(uid, pwd, uname) values (%s, '1234', %s);"
uid, uname = 'djy', '대조영'
cur.execute(sql_insert_ph, (uid, uname))
conn.commit()

- 여러건의 데이터를 placeholder를 이용해서 insert

In [27]:
users =(('gdhong', '홍길동'),('jbpark','박재범'))
cur.executemany(sql_insert_ph, users)
conn.commit()

In [28]:
# 강사가 추천하는 방법


users =(('gdhong2', '홍길동'),('jbpark2','박재범'))
for user in users:
    cur.execute(sql_insert_ph, user)

conn.commit()

## 마무리

In [29]:
cur.close()
conn.close()